# This notebook is supposed to work on Google Colab

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os

import tensorflow as tf
import numpy as np


cwd = os.path.join(os.getcwd(), 'drive', 'MyDrive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive', True)

Mounted at /content/drive


## Submission
1. Read masks
2. Prepare RLE
3. Create .zip file

In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/submissions/utils')

The submission file will be a zip file containing the *submission.json* file.
It must have the following structure:

 - image name (without any extension, e.g., png, jpg, etc.);
 - shape: shape of the original image as list [Height, Width];
 - team: team of the original image. One in {'Bipbip', 'Pead', 'Roseau', 'Weedelec'};
 - crop: crop of the original image. One in {'Haricot', 'Mais'};
 - segmentation: prediction (as a dict):
     - crop: RLE encoded crop segmentation (no weed);
     - weed: RLE encoded weed segmentation (no crop);

In [ ]:
teams = ['Bipbip', 'Pead', 'Roseau', 'Weedelec']
plants = ['Haricot', 'Mais']

We generate the RLE only for the dataset we trained, while for the others we generate fake masks

In [ ]:
# 1. Read the example RGB mask and transform it into integer labels
#                     +
# 2. Prepare .json file for the submission
from prepare_submission import rle_encode
from read_mask import read_rgb_mask
import json
from PIL import Image


submission_dict = {}
for team in teams:
    for plant in plants:
        print("Generating submission for {}/{}...".format(team, plant))
        if team is 'Bipbip' and plant is 'Haricot':
          files_dir = os.path.join(cwd, 'Development_Dataset', 'Test_Dev', team, plant, 'Masks')
          filenames = os.listdir(files_dir)

          for filename in filenames:
              mask = read_rgb_mask(os.path.join(files_dir, filename))
              filename = filename[:-4]
              submission_dict[filename] = {}
              submission_dict[filename]['shape'] = [mask.shape[0], mask.shape[1]]
              submission_dict[filename]['team'] = team
              submission_dict[filename]['crop'] = plant
              submission_dict[filename]['segmentation'] = {}

              # RLE encoding
              # crop
              rle_encoded_crop = rle_encode(mask == 1)
              # weed
              rle_encoded_weed = rle_encode(mask == 2)

              submission_dict[filename]['segmentation']['crop'] = rle_encoded_crop
              submission_dict[filename]['segmentation']['weed'] = rle_encoded_weed
        else:
            files_dir = os.path.join(cwd, 'Development_Dataset', 'Test_Dev', team, plant, 'Images')
            filenames = os.listdir(files_dir)

            for filename in filenames:
                img = Image.open(os.path.join(files_dir, filename))
                img_size = [img.size[1], img.size[0]]
                fake_mask = np.ones([img_size[0], img_size[1], 3])

                filename = filename[:-4]
                submission_dict[filename] = {}
                submission_dict[filename]['shape'] = img_size
                submission_dict[filename]['team'] = team
                submission_dict[filename]['crop'] = plant
                submission_dict[filename]['segmentation'] = {}

                # RLE encoding
                # crop
                rle_encoded_crop = rle_encode(fake_mask == 1)
                # weed
                rle_encoded_weed = rle_encode(fake_mask == 2)

                submission_dict[filename]['segmentation']['crop'] = rle_encoded_crop
                submission_dict[filename]['segmentation']['weed'] = rle_encoded_weed
        print("Finished submission for {}/{}".format(team, plant))
        print()

Generating submission for Bipbip/Haricot...
Finished submission for Bipbip/Haricot

Generating submission for Bipbip/Mais...
Finished submission for Bipbip/Mais

Generating submission for Pead/Haricot...
Finished submission for Pead/Haricot

Generating submission for Pead/Mais...
Finished submission for Pead/Mais

Generating submission for Roseau/Haricot...
Finished submission for Roseau/Haricot

Generating submission for Roseau/Mais...
Finished submission for Roseau/Mais

Generating submission for Weedelec/Haricot...
Finished submission for Weedelec/Haricot

Generating submission for Weedelec/Mais...
Finished submission for Weedelec/Mais



In [ ]:
# Let's see if the number of elements in the dictionary is equal to:
#       nr. test images (=15) * nr. datasets (=8) = 120
len(submission_dict.keys())

160

In [ ]:
save_to = os.path.join(cwd, 'submissions', 'predictions')
submission_filename = "prediction_" + str(len(os.listdir(save_to)))
json_file = 'submission.json'
json_path = os.path.join(save_to, json_file)

# Finally, save the results into the .json file
with open(json_file, 'w') as f:
    json.dump(submission_dict, f)

In [ ]:
# 3. Zip the .json file of step 2 and delete it from the directory
from zipfile import ZipFile

zip_path = os.path.join(save_to, (submission_filename + '.zip'))
with ZipFile(zip_path, 'w') as zip:
    zip.write(json_file)

# Delete the .json out of the .zip file
if os.path.exists(json_file):
    os.remove(json_file)